In [1]:
pip install mistralai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 7.0 MB/s eta 0:00:00


# single image


In [2]:
API_KEY='rrENEIP4CNCLjIyGyG9YGbgGXgMvcWPo'


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
def encode_image(path):
    with open(path, "rb") as img:
        return base64.b64encode(img.read()).decode("utf-8")

def extract_ocr_text(base64image):
    response = client.ocr.process(
        model="mistral-ocr-latest",
        document={
            "type": "image_url",
            "image_url": f"data:image/png;base64,{base64image}"
        }
    )
    return response.pages[0].markdown

In [10]:
import base64
from PIL import Image
import os
from mistralai import Mistral

API_KEY = 'rrENEIP4CNCLjIyGyG9YGbgGXgMvcWPo'
client = Mistral(api_key=API_KEY)

image_path = '/content/drive/MyDrive/UPS_datasets /ups_labels/L16_aug11.jpg'

original = Image.open(image_path)
width, height = original.size


upper_half = original.crop((0, 0, width, int(height * 0.6)))
upper_half_path = 'upper_half.png'
upper_half.save(upper_half_path)

lower_half = original.crop((0, int(height * 0.6), width, height))
lower_half_path = 'lower_half.png'
lower_half.save(lower_half_path)



upper_b64 = encode_image(upper_half_path)
lower_b64 = encode_image(lower_half_path)

upper_text = extract_ocr_text(upper_b64)
lower_text = extract_ocr_text(lower_b64)

combined_text = upper_text.strip() + "\n\n" + lower_text.strip()

# print("=== COMBINED OCR OUTPUT ===\n")
# print(combined_text)



In [11]:

import re

def clean_ocr_text(text):
    lines = text.splitlines()
    cleaned_lines = []

    for line in lines:
        line = line.strip()

        if not line:
            continue

        if line.startswith("![") and "](" in line:
            continue
        if set(line.replace('|', '').strip()) == {'-'}:
            continue

        if '|' in line:
            parts = [part.strip() for part in line.split('|') if part.strip()]
            line = ' '.join(parts)

        cleaned_lines.append(line)

    return '\n'.join(cleaned_lines)


cleaned_text = clean_ocr_text(combined_text)
# print("=== CLEANED OCR TEXT ===\n")
# print(cleaned_text)

os.remove(upper_half_path)
os.remove(lower_half_path)


normalized_text = re.sub(r"[*#]+", "", cleaned_text)

normalized_text = re.sub(r"(SHIP[\s_]*TO)[:\-]?", "SHIP TO", normalized_text, flags=re.IGNORECASE)

# print(normalized_text)

In [12]:
ship_to_match = re.search(
    r"SHIP TO\s*(.*?)(?:TRACKING|1Z|TILLING|COD|BILLING|REFERENCE|\Z)",
    normalized_text,
    re.IGNORECASE | re.DOTALL
)

if ship_to_match:
    ship_to_block = ship_to_match.group(1).strip()
    print("SHIP TO BLOCK:\n", ship_to_block)
else:
    print("SHIP TO BLOCK: Not found")


tracking_match = re.search(
    r"TRACKING\s*[:#]?\s*([A-Z0-9 ]{10,})",
    normalized_text,
    re.IGNORECASE
)

if tracking_match:
    tracking_number = tracking_match.group(1).strip()
    print("TRACKING NUMBER BLOCK:\n", tracking_number)
else:
    print("TRACKING NUMBER BLOCK: Not found")

SHIP TO BLOCK:
 JOHN SMITH
123 MAIN STREET
GREENWICH CT 06830
CT 069 9-01
UPS NEXT DAY AIR 1
TRACKING NUMBER BLOCK:
 12 112 F0 01 4588 0405


In [13]:
lines = ship_to_block.splitlines()
cleaned_lines = [line.strip() for line in lines if line.strip()]
ship_to_block_cleaned = '\n'.join(cleaned_lines)
print("CLEANED SHIP TO BLOCK:\n", ship_to_block_cleaned)


CLEANED SHIP TO BLOCK:
 JOHN SMITH
123 MAIN STREET
GREENWICH CT 06830
CT 069 9-01
UPS NEXT DAY AIR 1


In [14]:

import re

ship_to_lines = [line.strip() for line in ship_to_block_cleaned.strip().split('\n') if line.strip()]

name = ship_to_lines[0] if len(ship_to_lines) > 0 else ""
street = ship_to_lines[1] if len(ship_to_lines) > 1 else ""
city_state_zip_line = ship_to_lines[2] if len(ship_to_lines) > 2 else ""
possible_phone_line = ship_to_lines[3] if len(ship_to_lines) > 3 else ""

city = state = zip_code = phone = ""

# Try format: "CITY ST ZIP"
match_full = re.match(r"(.+?)\s+([A-Z]{2})\s+(\d{5})", city_state_zip_line)
if match_full:
    city, state, zip_code = match_full.groups()

# Else try: "CITY, ST"
elif re.match(r"(.+?),\s*([A-Z]{2})", city_state_zip_line):
    match = re.match(r"(.+?),\s*([A-Z]{2})", city_state_zip_line)
    city, state = match.groups()
    zip_code = None

# Detect phone number line
if len(re.sub(r"\D", "", possible_phone_line)) >= 10:
    phone = possible_phone_line.strip()
else:
    phone = None

# Output
print("NAME:", name)
print("STREET:", street)
print("CITY:", city)
print("STATE:", state)
print("ZIP CODE:", zip_code if zip_code else "None")
print("PHONE:", phone if phone else "None")
print("TRACKING ID:", tracking_number)



NAME: JOHN SMITH
STREET: 123 MAIN STREET
CITY: GREENWICH
STATE: CT
ZIP CODE: 06830
PHONE: None
TRACKING ID: 12 112 F0 01 4588 0405


# batch


In [19]:
API_KEY = 'rrENEIP4CNCLjIyGyG9YGbgGXgMvcWPo'
client = Mistral(api_key=API_KEY)

In [16]:
drive_folder = '/content/drive/MyDrive/labelss'

image_files = [os.path.join(drive_folder, f) for f in os.listdir(drive_folder)
               if f.lower().endswith(('.png', '.jpg', '.jpeg'))]


In [18]:
import os
import base64
import re
from PIL import Image
from mistralai import Mistral

def encode_image(path):
    with open(path, "rb") as img:
        return base64.b64encode(img.read()).decode("utf-8")

def extract_ocr_text(base64image):
    response = client.ocr.process(
        model="mistral-ocr-latest",
        document={
            "type": "image_url",
            "image_url": f"data:image/png;base64,{base64image}"
        }
    )
    return response.pages[0].markdown

In [20]:
def clean_ocr_text(text):
    lines = text.splitlines()
    cleaned_lines = []

    for line in lines:
        line = line.strip()
        if not line:
            continue
        if line.startswith("![") and "](" in line:
            continue
        if set(line.replace('|', '').strip()) == {'-'}:
            continue
        if '|' in line:
            parts = [part.strip() for part in line.split('|') if part.strip()]
            line = ' '.join(parts)
        cleaned_lines.append(line)

    return '\n'.join(cleaned_lines)

In [24]:
for image_path in image_files:
    original = Image.open(image_path)
    width, height = original.size

    upper_half = original.crop((0, 0, width, int(height * 0.6)))
    lower_half = original.crop((0, int(height * 0.6), width, height))

    upper_half_path = 'upper_half.png'
    lower_half_path = 'lower_half.png'
    upper_half.save(upper_half_path)
    lower_half.save(lower_half_path)

    upper_b64 = encode_image(upper_half_path)
    lower_b64 = encode_image(lower_half_path)

    upper_text = extract_ocr_text(upper_b64)
    lower_text = extract_ocr_text(lower_b64)

    combined_text = upper_text.strip() + "\n\n" + lower_text.strip()
    cleaned_text = clean_ocr_text(combined_text)

    os.remove(upper_half_path)
    os.remove(lower_half_path)

    normalized_text = re.sub(r"[*#]+", "", cleaned_text)
    normalized_text = re.sub(r"(SHIP[\s_]*TO)[:\-]?", "SHIP TO", normalized_text, flags=re.IGNORECASE)

    ship_to_match = re.search(
        r"SHIP TO\s*(.*?)(?:TRACKING|1Z|TILLING|COD|BILLING|REFERENCE|\Z)",
        normalized_text,
        re.IGNORECASE | re.DOTALL
    )

    if ship_to_match:
        ship_to_block = ship_to_match.group(1).strip()
        print(f"\nIMAGE: {os.path.basename(image_path)}")
        #print("SHIP TO BLOCK:\n", ship_to_block)
    else:
        print(f"\nIMAGE: {os.path.basename(image_path)}")
        print("SHIP TO BLOCK: Not found")
        continue

    tracking_match = re.search(
        r"TRACKING\s*[:#]?\s*([A-Z0-9 ]{10,})",
        normalized_text,
        re.IGNORECASE
    )

    if tracking_match:
        tracking_number = tracking_match.group(1).strip()
        #print("TRACKING NUMBER BLOCK:\n", tracking_number)
    else:
        print("TRACKING NUMBER BLOCK: Not found")
        tracking_number = "None"

    lines = ship_to_block.splitlines()
    cleaned_lines = [line.strip() for line in lines if line.strip()]
    ship_to_block_cleaned = '\n'.join(cleaned_lines)
    #print("CLEANED SHIP TO BLOCK:\n", ship_to_block_cleaned)

    ship_to_lines = [line.strip() for line in ship_to_block_cleaned.strip().split('\n') if line.strip()]

    name = ship_to_lines[0] if len(ship_to_lines) > 0 else ""
    street = ship_to_lines[1] if len(ship_to_lines) > 1 else ""
    city_state_zip_line = ship_to_lines[2] if len(ship_to_lines) > 2 else ""
    possible_phone_line = ship_to_lines[3] if len(ship_to_lines) > 3 else ""

    city = state = zip_code = phone = ""

    match_full = re.match(r"(.+?)\s+([A-Z]{2})\s+(\d{5})", city_state_zip_line)
    if match_full:
        city, state, zip_code = match_full.groups()
    elif re.match(r"(.+?),\s*([A-Z]{2})", city_state_zip_line):
        match = re.match(r"(.+?),\s*([A-Z]{2})", city_state_zip_line)
        city, state = match.groups()
        zip_code = None

    if len(re.sub(r"\D", "", possible_phone_line)) >= 10:
        phone = possible_phone_line.strip()
    else:
        phone = None

    print("NAME:", name)
    print("STREET:", street)
    print("CITY:", city)
    print("STATE:", state)
    print("ZIP CODE:", zip_code if zip_code else "None")
    print("PHONE:", phone if phone else "None")
    print("TRACKING ID:", tracking_number)



IMAGE: L16_aug11.jpg
NAME: JOHN SMITH
STREET: 123 MAIN STREET
CITY: GREENWICH
STATE: CT
ZIP CODE: 06830
PHONE: None
TRACKING ID: 12 112 F0 01 4588 0405

IMAGE: L6_aug20.jpg
TRACKING NUMBER BLOCK: Not found
NAME: KARTHIK NAIK
STREET: 9876543210
CITY: 
STATE: 
ZIP CODE: None
PHONE: None
TRACKING ID: None

IMAGE: L9_aug30.jpg
NAME: UPS Customer 2
STREET: 123 Main Street
CITY: Brussels
STATE: BE
ZIP CODE: None
PHONE: 1234567890
TRACKING ID: 1Y E59 512 34 5678 9012

IMAGE: L4_aug26.jpg
NAME: LANE MARTIN
STREET: 8801986582197
CITY: 
STATE: 
ZIP CODE: None
PHONE: None
TRACKING ID: 1Z X37 301 66 9076 5996

IMAGE: L5_aug28.jpg
NAME: DEVESH RAJARSHI 09123456789
STREET: MAIN STREET: GROUND FLOOR, 4TH CROSS ROAD, 2ND NEW YORK NY 10017
CITY: 
STATE: 
ZIP CODE: None
PHONE: None
TRACKING ID: 1Z 262 AY9 72 9860 3378

IMAGE: L3_aug29.jpg
NAME: JOSEPHINE MARIS
STREET: 147852369
CITY: 
STATE: 
ZIP CODE: None
PHONE: None
TRACKING ID: 1Z 1X0 894 68 3184 8881
